<a href="https://colab.research.google.com/github/rsalbertiaustral/labo2025v/blob/main/randomness_testRoxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/random/

Mounted at /content/drive


## Instalacion libreria NIST
Clono el repo que tiene los tests estadisticos de NIST
para los generadores de numeros pseudoaleatorios

*   https://github.com/terrillmoore/NIST-Statistical-Test-Suite
*   https://nvlpubs.nist.gov/nistpubs/legacy/sp/nistspecialpublication800-22r1a.pdf





### Clono el repo

que tiene los tests estadisticos de NIST para los generadores de numeros pseudoaleatorios

https://github.com/terrillmoore/NIST-Statistical-Test-Suite
https://nvlpubs.nist.gov/nistpubs/legacy/sp/nistspecialpublication800-22r1a.pdf

In [ ]:
!if cd  /content/NIST-Statistical-Test-Suite; then git pull; else git clone https://github.com/terrillmoore/NIST-Statistical-Test-Suite  /content/NIST-Statistical-Test-Suite; fi

/bin/bash: line 1: cd: /content/NIST-Statistical-Test-Suite: No such file or directory
Cloning into '/content/NIST-Statistical-Test-Suite'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 152 (delta 21), reused 21 (delta 11), pack-reused 113 (from 1)
Receiving objects: 100% (152/152), 47.41 MiB | 33.00 MiB/s, done.
Resolving deltas: 100% (45/45), done.


### instalaciones

In [ ]:
!cd /content/NIST-Statistical-Test-Suite && ./setup.sh && cd ./sts && make

Setting up directories in ./sts/experiments.
Created ./sts/obj.
Created ./sts/experiments/AlgorithmTesting.
Created ./sts/experiments/BBS.
Created ./sts/experiments/CCG.
Created ./sts/experiments/G-SHA1.
Created ./sts/experiments/LCG.
Created ./sts/experiments/MODEXP.
Created ./sts/experiments/MS.
Created ./sts/experiments/QCG1.
Created ./sts/experiments/QCG2.
Created ./sts/experiments/XOR.
Creating the subdirectories via ./sts/experiments/create-dir-script.
create-dir-script succeeded.
Directories are set up. Change directory to ./sts and say 'make'!
/usr/bin/gcc -o obj/assess.o -c ./src/assess.c
/usr/bin/gcc -o obj/frequency.o -c -Wall ./src/frequency.c
/usr/bin/gcc -o obj/blockFrequency.o -c -Wall ./src/blockFrequency.c
/usr/bin/gcc -o obj/cusum.o -c -Wall ./src/cusum.c
/usr/bin/gcc -o obj/runs.o -c -Wall ./src/runs.c
/usr/bin/gcc -o obj/longestRunOfOnes.o -c -Wall ./src/longestRunOfOnes.c
/usr/bin/gcc -o obj/serial.o -c -Wall ./src/serial.c
/usr/bin/gcc -o obj/rank.o -c -Wall ./src

## Ejecuto los Experimentos


### Creo el template de los experimentos



1.   0 = hay archivbo de entrada
2.   el archivo con la secuencia de bints
3.   1  aplicacion de TODOS los tests
4.   0  continuar
5.  10 la cantidad de streams a experimentar
6.   1 = archivo binario,  cada byte tiene 8 bits utiles y (supuestamente) random



In [ ]:
!echo "0" > /content/NIST-Statistical-Test-Suite/test/semilla.txt
!echo "/content/NIST-Statistical-Test-Suite/sts/data/semilla.bin"  >> /content/NIST-Statistical-Test-Suite/test/semilla.txt
!echo "1" >> /content/NIST-Statistical-Test-Suite/test/semilla.txt
!echo "0" >> /content/NIST-Statistical-Test-Suite/test/semilla.txt
!echo "10" >> /content/NIST-Statistical-Test-Suite/test/semilla.txt
!echo -n "1" >> /content/NIST-Statistical-Test-Suite/test/semilla.txt



In [ ]:
import  numpy as np
import  sympy
import  os
import  random

carpeta_raiz= "/content/NIST-Statistical-Test-Suite/sts/experiments/AlgorithmTesting/"


Establezco la cantidad de semillas que voy a crear
se crean en total 2*semillas_cantidad semillas


*   semillas_cantidad numeros primos del intervalo [100000, 999999]
*   0, 2, 4, 6, ..., 2*(semillas_cantidad - 1 )



In [ ]:
semillas_cantidad = 100
bits_cantidad = 40000000
semilla_experimento = 102191 # 410551 200177

### Genero los tests para las semillas numeros primos

quedan en la carpeta "/content/NIST-Statistical-Test-Suite/sts/experiments/AlgorithmTesting/$semilla_experimento

y se llaman  s_xxxxxx.txt

In [ ]:
# pruebo numeros primos
import  random
random.seed(semilla_experimento)  # inicilizo la libreria random
carpeta_primos = carpeta_raiz + str(semilla_experimento) + "/"
!mkdir -p $carpeta_primos
primos = random.sample( list(sympy.primerange(100000, 999999)), semillas_cantidad )

for i, sem  in  enumerate(primos):
  print( i, end=' ')
  archivo_test = carpeta_primos + "s_" + str(sem).rjust(6,'0') + ".txt"
  if ( False == os.path.exists(archivo_test) ):
    # genero vector de 10000000  enteros sin signo, aleatorios
    np.random.seed(sem)  # inicializo numpy
    v = np.random.randint(0, high=2**32, size=bits_cantidad//4, dtype=np.uint32)
    arch=open("/content/NIST-Statistical-Test-Suite/sts/data/semilla.bin","wb")
    arch.write(v)
    arch.close()

    # ejecuto los tests
    !cd /content/NIST-Statistical-Test-Suite/sts/ && cat /content/NIST-Statistical-Test-Suite/test/semilla.txt | /content/NIST-Statistical-Test-Suite/sts/assess 100000  >/dev/null 2>&1
    # copio la salida que queda en finalAnalysisReport.txt  a un archivo con nombre de la semilla
    linea = "cp " + carpeta_raiz + "finalAnalysisReport.txt " + archivo_test
    !$linea


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

### Ahora genero los tests para los numeros PARES

In [ ]:
# pruebo numeros pares consecutivos 0, 2, 4, 6, ...
for sem in range(0,2*semillas_cantidad,2):
  print( sem, end=' ')
  archivo_test = carpeta_raiz + "s_" + str(sem).rjust(6,'0') + ".txt"
  if ( False == os.path.exists(archivo_test) ):
    np.random.seed(sem)   # inicializo numpy

    # genero vector de 10000000  enteros sin signo, aleatorios
    v = np.random.randint(0, high=2**32, size=bits_cantidad//4, dtype=np.uint32)
    arch=open("/content/NIST-Statistical-Test-Suite/sts/data/semilla.bin","wb")
    arch.write(v)
    arch.close()

    # ejecuto los tests
    !cd /content/NIST-Statistical-Test-Suite/sts/ && cat /content/NIST-Statistical-Test-Suite/test/semilla.txt | /content/NIST-Statistical-Test-Suite/sts/assess 100000   >/dev/null 2>&1
    # copio la salida que queda en finalAnalysisReport.txt  a un archivo con nombre de la semilla
    linea = "cp " + carpeta_raiz + "finalAnalysisReport.txt " + archivo_test
    !$linea


0 2 4 6 8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44 46 48 50 52 54 56 58 60 62 64 66 68 70 72 74 76 78 80 82 84 86 88 90 92 94 96 98 100 102 104 106 108 110 112 114 116 118 120 122 124 126 128 130 132 134 136 138 140 142 144 146 148 150 152 154 156 158 160 162 164 166 168 170 172 174 176 178 180 182 184 186 188 190 192 194 196 198 

## Analizo los Experimentos

> Añadir blockquote



### uno todas las salidas, la de los primos y la de los pares

In [ ]:
archivo_salida =  carpeta_raiz + "todos_" + str(semilla_experimento) + ".txt"


In [ ]:
!cd $carpeta_raiz && grep  -E "[0-9]+\.[0-9]+" s_*.txt | grep -v "\*" | sed 's/  */ /g'> $archivo_salida
!cd $carpeta_primos && grep  -E "[0-9]+\.[0-9]+" s_*.txt | grep -v "\*" | sed 's/  */ /g'>> $archivo_salida

### Analizo la salida

FUNDAMENTAL : p-values mayores significa mejor aleatoriedad

For these tests, each P-value is the probability that a perfect random number generator would have produced a sequence less random than the sequence that was tested, given the kind of nonrandomness assessed by the test. If a P-value for a test is determined to be equal to 1, then the sequence appears to have perfect randomness. A P-value of zero indicates that the sequence appears to be completely non-random.

In [ ]:
import polars as pl


dataset = pl.read_csv(
  source = archivo_salida,
  separator=" ",
  has_header=False,
  truncate_ragged_lines=True)

# me quedo con las columnas que me interesan
dataset = dataset.select(pl.nth([0, 11, 13]))
dataset.columns = ["archivo", "pvalor", "analisis_tipo"]

# extraigo la semilla del nombre del archivo
dataset = dataset.with_columns(
    pl.col("archivo").str.slice(2, 6).cast(pl.Int32).alias("semilla")
)

# si es impar, en ESTE caso, es el numero prinmero
dataset = dataset.with_columns((pl.col("semilla") % 2).alias("semilla_prima"))

# genero la tabla con la agrupacion
tbl = (
    dataset
    .group_by(["analisis_tipo", "semilla_prima"])
    .agg([
        pl.len().alias("cantidad"),
        pl.col("pvalor").mean().alias("mean_pvalor")
    ])
)

# Sort by 'analisis' and 'pos'
tbl = tbl.sort(["analisis_tipo", "semilla_prima"])

with pl.Config(tbl_rows=-1):
    print(tbl)

tbl.write_csv("/content/drive/MyDrive/random/"+"tbl_"+str(semilla_experimento)+".csv",
    separator=",")

shape: (24, 4)
┌────────────────────────┬───────────────┬──────────┬─────────────┐
│ analisis_tipo          ┆ semilla_prima ┆ cantidad ┆ mean_pvalor │
│ ---                    ┆ ---           ┆ ---      ┆ ---         │
│ str                    ┆ i32           ┆ u32      ┆ f64         │
╞════════════════════════╪═══════════════╪══════════╪═════════════╡
│ ApproximateEntropy     ┆ 0             ┆ 100      ┆ 0.456144    │
│ ApproximateEntropy     ┆ 1             ┆ 100      ┆ 0.494425    │
│ BlockFrequency         ┆ 0             ┆ 99       ┆ 0.459401    │
│ BlockFrequency         ┆ 1             ┆ 100      ┆ 0.541946    │
│ CumulativeSums         ┆ 0             ┆ 200      ┆ 0.450535    │
│ CumulativeSums         ┆ 1             ┆ 200      ┆ 0.51431     │
│ FFT                    ┆ 0             ┆ 100      ┆ 0.536846    │
│ FFT                    ┆ 1             ┆ 99       ┆ 0.505538    │
│ Frequency              ┆ 0             ┆ 100      ┆ 0.50492     │
│ Frequency              ┆ 1     


A mayor p-valor,  mejor propiedades tiene la secuencia de numeros aleatorios

semilla_prima = 1  indica que la semilla se inicializó con un numero prino

Se observa que para este anlaisis que, por poca diferenica, las semillas numeros primos generan una secuencia un poco mejor de numeros aleatorios
particularmente para el Linear Complexity

